# Импорт модулей

In [17]:
import json
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import log_loss

# Загрузка данных

Тренировочные данные состоят из полей 'text_id', 'text' и 11 полей с таргетами. Не стоит забывать, что может быть больше одной болезни для каждого случая.

Тестовые же данные содержат поля 'text_id' и 'text'.

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head(2)

,text_id,text,эймериоз,абсцесс,диспепсия молодняка,остертагиоз,мастит,инфекционный ринотрахеит,отёк вымени,тенденит,сибирская язва,лишай,другое
0,0,"Корова, видимо вставая, раздавила себе сосок. ...",0,0,0,0,1,0,0,0,0,0,0
1,1,Корове 8 лет! Месяц назад промеж четвертей вым...,0,0,0,0,0,0,0,0,1,1,1


In [5]:
test.head(2)

,text_id,text
0,294,Понос у месячных телят. Подскажите методы и сп...
1,295,"Понос у телят, чем лечить? \nЧем можно вылечит..."


# Базовая модель

В качестве базового решения используется CatBoostClassifier, поддерживающий текстовые фичи. Помимо этого задача является мультилейбл классификацией, поэтому модель обернута в OneVsRestClassifier.

**Делим данные на тренировочную и валидационную выборку**

In [6]:
X_train, X_val, y_train, y_val = train_test_split(pd.DataFrame(train['text']), train[train.columns[2:]], test_size=0.2)

**Обучаем модель**

In [7]:
model = OneVsRestClassifier(estimator=CatBoostClassifier(iterations = 100, text_features=['text'], 
                                                         verbose = 50, allow_writing_files=False))
model.fit(X_train, y_train)

Learning rate set to 0.045854
0:	learn: 0.6748344	total: 81.7ms	remaining: 8.09s
50:	learn: 0.3577107	total: 1.55s	remaining: 1.49s
99:	learn: 0.2894234	total: 3.19s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6754896	total: 30.2ms	remaining: 2.99s
50:	learn: 0.4102976	total: 1.82s	remaining: 1.75s
99:	learn: 0.3551146	total: 3.27s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6751149	total: 23.5ms	remaining: 2.33s
50:	learn: 0.3688462	total: 1.58s	remaining: 1.52s
99:	learn: 0.3136273	total: 3.08s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6726563	total: 20.6ms	remaining: 2.04s
50:	learn: 0.3554416	total: 1.68s	remaining: 1.61s
99:	learn: 0.3034627	total: 3.24s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6742050	total: 34.8ms	remaining: 3.44s
50:	learn: 0.3478183	total: 1.52s	remaining: 1.46s
99:	learn: 0.2806680	total: 2.98s	remaining: 0us
Learning rate set to 0.045854
0:	learn: 0.6706879	total: 28.6ms	remaining: 2.83s
50:	learn: 0.3

OneVsRestClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7fca2034ac10>)

# Считаем метрику

В качестве "gt" (ground truth) функция принимает на вход датафрейм/массив из 10 столбцов (все классы, кроме "другое"). Предсказанные значения – "pr", также должны быть либо в виде датафрейма, либо в виде массива.

In [8]:
def log_loss_score(gt, pr):
    
    log_loss_ = 0
    
    gt = np.array(gt)
    
    for i in range(10):
        log_loss_ += log_loss(gt[:, i], pr[:, i])
        
    return log_loss_ / 10

In [9]:
log_loss_score(y_val, model.predict_proba(X_val))

0.3409663042406871

# Создание файла отправки

В файле с ответами должны быть вероятности для каждого класса. Сумма вероятностей в каждой строке может быть больше 1.

In [10]:
submission_columns = ['text_id'] + list(train.columns[2:-1])
submission = pd.concat([test['text_id'], pd.DataFrame(model.predict_proba(pd.DataFrame(test['text']))[:, :10])], axis=1)
submission.columns = submission_columns

In [11]:
submission.head()

,text_id,эймериоз,абсцесс,диспепсия молодняка,остертагиоз,мастит,инфекционный ринотрахеит,отёк вымени,тенденит,сибирская язва,лишай
0,294,0.062719,0.113136,0.072705,0.074253,0.103621,0.075186,0.066241,0.057393,0.104223,0.061639
1,295,0.567817,0.143268,0.392205,0.375057,0.211739,0.133588,0.136544,0.406265,0.173632,0.088913
2,296,0.198515,0.120845,0.369088,0.180145,0.233022,0.078119,0.065792,0.311344,0.104504,0.063568
3,297,0.065979,0.148187,0.077228,0.087859,0.083386,0.069549,0.060314,0.062278,0.103982,0.068143
4,298,0.068813,0.138642,0.084449,0.083486,0.188230,0.067497,0.089238,0.067269,0.115569,0.089609


In [16]:
submission_json = {str(k): {"span": list(), "label": list(v.values())} \
                   for k,v in submission.set_index('text_id').to_dict('index').items()}

submission_json['294']

{'span': [],
 'label': [0.0627194363685613,
  0.11313570529902758,
  0.07270482269352871,
  0.0742534689467756,
  0.10362104310746847,
  0.07518588621133018,
  0.06624119018743685,
  0.05739265274755671,
  0.10422273080497868,
  0.061638782036372346]}

In [18]:
with open('sample_submission.json', 'w') as final_submit:
    json.dump(submission_json, final_submit, indent=4)